In [1]:
using Pkg; Pkg.activate("/home/jovyan/NFMOneNotebook")
using GenericRofReader
using Plots
using Logging
using Statistics
Logging.disable_logging(Logging.Warn)

  Activating project at `~/NFMOneNotebook`


LogLevel(1001)

In [2]:
# Define the struct FMDStatusMessage containing the status of FMD for all four cameras
struct ViperFMDStatusMessage
    fmd_status_cam_rear::Vector{Int32}
    fmd_status_cam_left::Vector{Int32}
    fmd_status_cam_front::Vector{Int32}
    fmd_status_cam_right::Vector{Int32}

    #constructor to create empty instance of the struct
    function ViperFMDStatusMessage()
        new(Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}()
        )
    end
end

struct ViperFMDStatusMessage4
    # FMD Status
    fmd_status_cam_rear::Vector{Int32}
    fmd_status_cam_left::Vector{Int32}
    fmd_status_cam_front::Vector{Int32}
    fmd_status_cam_right::Vector{Int32}

    # Mean root dev 
    mean_rot_dev_cam_rear::Vector{Tuple{Float32, Float32, Float32}}
    mean_rot_dev_cam_left::Vector{Tuple{Float32, Float32, Float32}}
    mean_rot_dev_cam_front::Vector{Tuple{Float32, Float32, Float32}}
    mean_rot_dev_cam_right::Vector{Tuple{Float32, Float32, Float32}}

    #constructor to create empty instance of the struct
    function ViperFMDStatusMessage4()
        new(Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}(),
            Vector{Tuple{Float32, Float32, Float32}}(), 
            Vector{Tuple{Float32, Float32, Float32}}(), 
            Vector{Tuple{Float32, Float32, Float32}}(),  
            Vector{Tuple{Float32, Float32, Float32}}()
        )
    end
end

function print_fmd_message(fmdMsg::ViperFMDStatusMessage4)
        println("fmd_status_cam_rear ...")
        println(fmdMsg.fmd_status_cam_rear)

        println("fmd_status_cam_left ...")
        println(fmdMsg.fmd_status_cam_left)

        println("fmd_status_cam_front ...")
        println(fmdMsg.fmd_status_cam_front)

        println("fmd_status_cam_right ...")
        println(fmdMsg.fmd_status_cam_right)

        println("mean_rot_dev_cam_rear ...")
        println(fmdMsg.mean_rot_dev_cam_rear)
        
        println("mean_rot_dev_cam_left ...")
        println(fmdMsg.mean_rot_dev_cam_left)
      
        println("mean_rot_dev_cam_front ...")
        println(fmdMsg.mean_rot_dev_cam_front)
      
        println("mean_rot_dev_cam_right ...")
        println(fmdMsg.mean_rot_dev_cam_right)
    
end

function parse_viper_fmd_status(interface_samples)::ViperFMDStatusMessage4
    parsed_data = ViperFMDStatusMessage4()

    #println("parsed data obj created")
    try
        for element in interface_samples
            push!(parsed_data.fmd_status_cam_rear, element.m_status_cam_rear)
            push!(parsed_data.fmd_status_cam_left, element.m_status_cam_left)
            push!(parsed_data.fmd_status_cam_front, element.m_status_cam_front)
            push!(parsed_data.fmd_status_cam_right, element.m_status_cam_right)

            mytuple = element.m_meta_data_cam_rear.m_mean_rot_dev.m_inheritedStructure.m_data.m_inheritedStructure.m_value.data
            push!(parsed_data.mean_rot_dev_cam_rear, mytuple)
            mytuple = element.m_meta_data_cam_left.m_mean_rot_dev.m_inheritedStructure.m_data.m_inheritedStructure.m_value.data
            push!(parsed_data.mean_rot_dev_cam_left, mytuple)
            mytuple = element.m_meta_data_cam_front.m_mean_rot_dev.m_inheritedStructure.m_data.m_inheritedStructure.m_value.data
            push!(parsed_data.mean_rot_dev_cam_front, mytuple)
            mytuple = element.m_meta_data_cam_right.m_mean_rot_dev.m_inheritedStructure.m_data.m_inheritedStructure.m_value.data
            push!(parsed_data.mean_rot_dev_cam_right, mytuple)
        end
    catch e
        println("exception: $e")
        @error("An error occurred in parsing viper fmd status data: $e", "exit")
    end
    return parsed_data
end

parse_viper_fmd_status (generic function with 1 method)

In [3]:
using OneParkingReaders

#using OneParkingReaders.General: get_interface_reader_generic
using OneParkingReaders.Utils: get_interface_reader_generic

function extract_FMDstatus(rof_folder_path)::ViperFMDStatusMessage4
    
    interface_name = "FMDStatusMessage.c2w__fast_misalignment_status.FMDStatusMessage"

    println(interface_name)

    fullpath = joinpath(rof_folder_path, interface_name)

    reader = get_interface_reader_generic(fullpath; create_enums = false)
    
    all_valid_interface_samples = read_all_valid_interfaces(reader)
    
    print("length of all_valid_interface_samples: ") 
    println(length(all_valid_interface_samples))

    fmd_status_message = parse_viper_fmd_status(all_valid_interface_samples)

    return fmd_status_message
end

ldf is not supported
xls is not supported
xlsx is not supported


extract_FMDstatus (generic function with 1 method)

In [10]:
@enum CameraId begin
    camera_rear
    camera_left
    camera_front
    camera_right
end

# Initialize the KPIs
kpis = Dict{String, Union{Missing, Float64}}(
    "fmd_status_good_rate_rear" => missing,
    "fmd_status_good_rate_left" => missing,
    "fmd_status_good_rate_front" => missing,
    "fmd_status_good_rate_right" => missing
)

# Initialize the FMD Status
fmd_status = Dict{String, Union{Missing, Int32}}(
    "rear" => missing,
    "left" => missing,
    "front" => missing,
    "right" => missing
)

# Initialize the FMD Status
fmd_status_val = Dict{String, Union{Missing, Int32}}(
    "rear" => missing,
    "left" => missing,
    "front" => missing,
    "right" => missing
)

Dict{String, Union{Missing, Int32}} with 4 entries:
  "left"  => missing
  "right" => missing
  "rear"  => missing
  "front" => missing

In [5]:
function fmd_status_good_rate_calculator!(kpis, cam_str, fmd_status_message, expected_fmd_status)
    # Calculates the good rate percentage of the fmd status going backwards

    # println("start of the fmd_status_good_rate_calculator ... ")
    # println("cam_str: $(cam_str)")
    
    fmd_status_cam = getfield(fmd_status_message, Symbol("fmd_status_cam_", cam_str))
    expected_fmd_status_cam = expected_fmd_status[cam_str]

    # Initialize the index to the last element of the vector and count to zero
    index = length(fmd_status_cam)
    count_good_frames_from_end::Int = 0

    # println("fmd_status_current_cam: $(fmd_status_cam)")
    # println(expected_fmd_status_cam)

    #fmd_status_val[cam_str] = last(fmd_status_cam)

    while(index > 0)
        if(fmd_status_cam[index] != expected_fmd_status_cam)
            break
        end
        count_good_frames_from_end += 1
        index -= 1
    end

    println("count_good_frames_from_end : $(count_good_frames_from_end)")

    fmd_status_good_rate::Float64 =
        count_good_frames_from_end / length(fmd_status_cam) * 100

    fmd_status_good_rate = round(fmd_status_good_rate, digits=2)

    if (count_good_frames_from_end == 0)
        # All angles of the camera are NOT within the acceptable range for all frames
        @warn("camera: $cam_str, fmd_status_good_rate: $(fmd_status_good_rate) % ,
         all angles of the camera are NOT within the acceptable range at last frame")
    end

    println("fmd_status_good_rate: $(fmd_status_good_rate)")

    kpis["fmd_status_good_rate_" * cam_str] = fmd_status_good_rate
    
end

fmd_status_good_rate_calculator! (generic function with 1 method)

In [6]:
# ------------------- KPIs using FMD Status Message Data  -------------------------------
#outlierSequence::Bool = false

#Define the variants of badly mounted vehicle
bad_case_vehicle_list = ["USB09", "USB13", "DEB06"]


3-element Vector{String}:
 "USB09"
 "USB13"
 "DEB06"

In [7]:
function extract_vehicle_variant(file_path::String)
    # Split the file path by underscores
    parts = split(file_path, '_')
    
    if length(parts) >= 2
        return parts[2]
    else
        return "No vechile match found in $(file_path)"
    end
end

function isOutlierSequence(vehicle_variant::String, string_list::Vector{String})::Bool
    # Check if the target_string is in the string_list
    return vehicle_variant in bad_case_vehicle_list
end

function updateExpectedFMDStatus!(expected_fmd_status, vehicle_variant::String)

    # This sequence has atleast one of the camera badly mounted
    if(vehicle_variant == "DEB06")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 4
            expected_fmd_status["left"]  = 4
            expected_fmd_status["front"] = 3
            expected_fmd_status["right"] = 4
    
    elseif(vehicle_variant == "USB09")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 4
            expected_fmd_status["left"]  = 3
            expected_fmd_status["front"] = 4
            expected_fmd_status["right"] = 4

    elseif(vehicle_variant == "USB13")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 3
            expected_fmd_status["left"]  = 3
            expected_fmd_status["front"] = 4
            expected_fmd_status["right"] = 3
    end

end

function calFMDStatusMeanRootStdDev(fmd_status_message)::Dict{String, Any}

    threshold = [0.07219138, 0.06731955, 0.06731955] # radians

    cameras = ["rear", "left", "front", "right"]

    for cam_str in cameras
        rear_tuple = last(getfield(fmd_status_message, Symbol("mean_rot_dev_cam_", cam_str)))
       
        # println("last tuple m_mean_rot_dev in $(cam_str)  ...")
        # println(rear_tuple)
    
        # Check if the absolute values of the vector exceed the thresholds
        any_exceeds_threshold = any(abs.(rear_tuple) .> threshold)
     
        if(any_exceeds_threshold)
            fmd_status_val[cam_str]  = 3
            #println("$(cam_str) deadjusted")
        else
            fmd_status_val[cam_str]  = 4
            #println("$(cam_str) good")
        end
    end

    #println(fmd_status_val)

    return fmd_status_val
    
end

calFMDStatusMeanRootStdDev (generic function with 1 method)

In [8]:
using CSV
using DataFrames

function append_row(file_path::String, row_data::Any)

    df = DataFrame(row_data)
    
    if isfile(file_path)
        # Append mode: Write without header
        CSV.write(file_path, df, append=true, header=false)
    else
        # Create new file or overwrite existing file
        CSV.write(file_path, df)
    end
end


function isGoodORBad(good_rate_cam)
    if(good_rate_cam > 0.0)
         return 1 #"Correct"
    else 
        return 0 #"InCorrect"
    end
end

function areAllGoodORBad(kpis)
    if ( kpis["fmd_status_good_rate_rear"] > 0 &&  kpis["fmd_status_good_rate_left"] > 0 && kpis["fmd_status_good_rate_front"] > 0  && kpis["fmd_status_good_rate_right"] > 0)
        return 1 #"Correct"
    else 
        return 0 #"InCorrect"
    end
end

function isCorrectStatus(computed_status_val, expected_status_val, cam_str)
    if(computed_status_val[cam_str] == expected_status_val[cam_str] )
         return 1 #"Correct"
    else 
        return 0 #"InCorrect"
    end
end

function isCorrectStatusAll(computed_status_val, expected_status_val)
    
    cameras = ["rear", "left", "front", "right"]

    isFMDStatusOkay = 1

    for cam_str in cameras
        if(computed_status_val[cam_str] != expected_status_val[cam_str] )
            isFMDStatusOkay = 0
            break
        end
    end

    return isFMDStatusOkay
end

isCorrectStatusAll (generic function with 1 method)

In [13]:
using Glob

#hash_foc_run = "5949953b4aa70f36221f5ed345e7fb9c9c71cfff796cec4d04e7a91becfd9979"

hash_foc_run = "8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace"  #USB13
#hash_foc_run = "e97701cfb82df2cf8c895d4d1254f785161e109c20e797f93250fe0c54a94ba6"   #DEB06
#hash_foc_run = "16dafef3f75518ec02795c8e49b0174736a298eed579a5c18aea9355954f1edd"   #DEB05

rof_base_dir = joinpath("../data/ReadOnly/dyperdrsprod/ford-dat-3", hash_foc_run, "recompute_output")

# good : All angles of the camera are within the acceptable range
expected_fmd_status = Dict("rear" => 4, "left" => 4, "front" => 4, "right" => 4)

# Define the CSV file path
csv_file_path = "kpis_fmd_status_cal_with_metadata.csv"

function calculate_fmd_status(base_dir::String)
    # Define the patterns
    patterns = ["*.mf4/*.rof"]

    # Loop through the patterns and find matching files
    for pattern in patterns
        # Use glob to find files matching the pattern in the base directory
        for rof_folder_path in glob(joinpath(base_dir, pattern))
            
            println("rof_folder_path ........................................................................................... ")
            println(rof_folder_path)

            # Split the string by '/'
            parts = split(rof_folder_path, '/')
            
            # Extract the hash and file path
            hash_value = parts[6]       # The hash value
            # Extract the part of the file path before the extension
            file_path_with_extension = split(parts[8], '.')[1]
            file_path::String = file_path_with_extension
            
            println("File Path: $file_path")

            vehicle_variant::String = extract_vehicle_variant(file_path)

            println(vehicle_variant)

            if(isOutlierSequence(vehicle_variant, bad_case_vehicle_list))
                updateExpectedFMDStatus!(expected_fmd_status, vehicle_variant)
            end

            fmd_status_struct = extract_FMDstatus(rof_folder_path)

            # compute the FMD Status based on the FMD metadata
            fmd_status_val = calFMDStatusMeanRootStdDev(fmd_status_struct)

            # print_fmd_message(fmd_status_message)

            for cam_id in instances(CameraId)
            
                # Get camera string.
                cam_str::AbstractString = split(string(cam_id), "_", limit = 2)[2]
    
                # fmd_status_good_rate_calculator!(kpis, cam_str, fmd_status_struct, expected_fmd_status)

                # good_rate_cam = kpis["fmd_status_good_rate_" * cam_str ]

                fmd_status[cam_str] = isCorrectStatus(fmd_status_val, expected_fmd_status, cam_str)

                println("fmd_status[cam_str]: $(fmd_status[cam_str]) ")
       
            end

            status_all_cam = isCorrectStatusAll(fmd_status_val, expected_fmd_status)
                   
            row = Dict(
                    :Hash => hash_value, 
                    :Sequence => file_path,
                    :FMD_status_rear => fmd_status["rear"],
                    :FMD_status_left => fmd_status["left"], 
                    :FMD_status_front => fmd_status["front"],   
                    :FMD_status_right => fmd_status["right"], 
                    :FMD_status_all_cam => status_all_cam,
                    :FMD_status_last_rear => fmd_status_val["rear"],
                    :FMD_status_last_left => fmd_status_val["left"], 
                    :FMD_status_last_front => fmd_status_val["front"],   
                    :FMD_status_last_right => fmd_status_val["right"]
               )
                append_row(csv_file_path, row)
        end
    end
end

calculate_fmd_status(rof_base_dir)

rof_folder_path ........................................................................................... 
../data/ReadOnly/dyperdrsprod/ford-dat-3/8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace/recompute_output/FD3_USB13_20240717_190239_001.mf4/recall_FD3_USB13_20240717_190239_001.rof
File Path: FD3_USB13_20240717_190239_001
USB13
FMDStatusMessage.c2w__fast_misalignment_status.FMDStatusMessage
length of all_valid_interface_samples: 1116
fmd_status[cam_str]: 1 
fmd_status[cam_str]: 1 
fmd_status[cam_str]: 1 
fmd_status[cam_str]: 0 
rof_folder_path ........................................................................................... 
../data/ReadOnly/dyperdrsprod/ford-dat-3/8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace/recompute_output/FD3_USB13_20240717_190511_001.mf4/recall_FD3_USB13_20240717_190511_001.rof
File Path: FD3_USB13_20240717_190511_001
USB13
FMDStatusMessage.c2w__fast_misalignment_status.FMDStatusMessage
length of all_valid_inte